# FDR Correction
The folowing code does the following:
* Reads the p-value significance matrix
* Performs the FDR correction on it
* Stores the fdr corrected q-value brain file as a functional file  

In [12]:
from scipy import stats
import numpy as np
import os
from os.path import join as opj
from multiprocessing import Pool
import multiprocessing.managers
from functools import partial
import nibabel as nib


In [13]:
# read the significance matrix
X = np.load('/home1/varunk/Autism-Connectome-Analysis-bids-related/significance_matrix.npy', mmap_mode='r')
# X = np.load('/home1/ee3140506/funcConn/funcConnGUI/Scripts/Pvals.npy', mmap_mode = 'r')

# stack a column indentifying the roi value
X = np.c_[1:X.shape[0]+1,X]
num_ROIs = X.shape[0]

In [14]:
# For passing on the shared file:
class MyManager(multiprocessing.managers.BaseManager):
    pass
MyManager.register('np_zeros', np.zeros, multiprocessing.managers.ArrayProxy)

In [15]:
# Read mask files:
mask_file = os.path.expandvars('$FSLDIR/data/standard/MNI152_T1_2mm_brain_mask.nii.gz')
mask_data = nib.load(mask_file)
mask = mask_data.get_data()

# read the file on which you want to write the p-values
brain_file = '/home1/varunk/data/ABIDE-BIDS-Preprocessed/NYU/sub-0050952/func/sub-0050952_task-rest_run-1_bold.nii.gz'
brain_data = nib.load(brain_file)
brain = brain_data.get_data()

# Edit header to store ROI correlation maps in t(4th) dimension

(brain_data.header).set_data_shape([91,109,91,num_ROIs])

In [16]:
def do_fdr(fdrcorrected_brain_4Dtensor,fdrcorrected_brain_4Dtensor_mask, fdr_brain_voxel_list, brain_voxel_list):
    mask_data = nib.load(mask_file)
    mask = mask_data.get_data()
    
    # to use the deader of an anatomical file tht has float datatype. mask had uint datatype that made all the p-values zero
    brain_file = '/home1/varunk/data/ABIDE-BIDS-Preprocessed/NYU/sub-0050952/anat/sub-0050952_T1w.nii.gz'
    brain_data = nib.load(brain_file)
    
    
    # Two empty brain to store the fdr corrected values and mask (which voxel is correlated)
    brain = brain_data.get_data()
    
    brain_corrected_mask = np.zeros((brain.shape))


    roi_number = int(brain_voxel_list[0]) - 1
    brain_voxel_list = brain_voxel_list[1:]
    
    # FDR Correction: http://www.statsmodels.org/devel/generated/statsmodels.sandbox.stats.multicomp.fdrcorrection0.html#statsmodels.sandbox.stats.multicomp.fdrcorrection0
    
    alpha = 0.05
    from statsmodels.sandbox.stats.multicomp import fdrcorrection0

    brain_voxel_list_mask, brain_voxel_list  = fdrcorrection0(brain_voxel_list,alpha)
    
    fdr_brain_voxel_list[roi_number,:] =  brain_voxel_list
    
    
    
    x_dim, y_dim, z_dim = mask.shape

    brain_voxel_counter = 0
    for i in range(x_dim):
        for j in range(y_dim):
            for k in range(z_dim):
                if mask[i,j,k] == 1:
                    brain[i,j,k] = brain_voxel_list[brain_voxel_counter]
                    if brain_voxel_list_mask[brain_voxel_counter] == True:
                        brain_corrected_mask[i,j,k] = 1.0
                    brain_voxel_counter = brain_voxel_counter + 1
                    

    
    
    if brain_voxel_counter == len(brain_voxel_list):
        
     
    
        print('Brain shape ',brain.shape)
        fdrcorrected_brain_4Dtensor[:,:,:,roi_number] = brain
        fdrcorrected_brain_4Dtensor_mask[:,:,:,roi_number] = brain_corrected_mask

        print('Stored ROI file the shared arrays! ')
        print("Job Done for ROI-",roi_number)
      
        return 1
    else:
        print("p-values fitting failed for ROI-",roi_number)
        return 0

In [17]:
%%time
# Create pool of 8 workers

pool_inputs = [] #np.arange(number_of_ROIs)

for roi in range(num_ROIs):
    pool_inputs.append(X[roi,:])
    

# print (brain_data.header)


m = MyManager()
m.start()
fdrcorrected_brain_4Dtensor = m.np_zeros((brain_data.header.get_data_shape()))
fdrcorrected_brain_4Dtensor_mask = m.np_zeros((brain_data.header.get_data_shape()))
fdr_brain_voxel_list = m.np_zeros((X.shape[0],X.shape[1]-1))

func = partial(do_fdr, fdrcorrected_brain_4Dtensor,fdrcorrected_brain_4Dtensor_mask)

pool = Pool(8)

data_outputs = pool.map(func, pool_inputs)


Brain shape  (91, 109, 91)
Brain shape  (91, 109, 91)
Brain shape  (91, 109, 91)
Brain shape  (91, 109, 91)
Brain shape  (91, 109, 91)
Brain shape  (91, 109, 91)
Brain shape  (91, 109, 91)
Brain shape  (91, 109, 91)
Stored ROI file the shared arrays! 
Stored ROI file the shared arrays! 
Stored ROI file the shared arrays! 
Stored ROI file the shared arrays! 
Stored ROI file the shared arrays! 
Stored ROI file the shared arrays! 
Job Done for ROI- 54
Job Done for ROI- 18
Job Done for ROI- 9
Job Done for ROI- 63
Job Done for ROI- 0
Stored ROI file the shared arrays! 
Job Done for ROI- 27
Stored ROI file the shared arrays! 
Job Done for ROI- 45
Job Done for ROI- 36
Brain shape  (91, 109, 91)
Stored ROI file the shared arrays! 
Job Done for ROI- 28
Brain shape  (91, 109, 91)
Brain shape  (91, 109, 91)
Stored ROI file the shared arrays! 
Stored ROI file the shared arrays! 
Job Done for ROI- 46
Job Done for ROI- 1
Brain shape  (91, 109, 91)
Brain shape  (91, 109, 91)
Brain shape  (91, 109, 91

Stored ROI file the shared arrays! 
Brain shape  (91, 109, 91)
Job Done for ROI- 111
Brain shape  (91, 109, 91)
Stored ROI file the shared arrays! 
Stored ROI file the shared arrays! 
Stored ROI file the shared arrays! 
Job Done for ROI- 84
Job Done for ROI- 93
Job Done for ROI- 75
Brain shape  (91, 109, 91)
Stored ROI file the shared arrays! 
Job Done for ROI- 129
Brain shape  (91, 109, 91)
Stored ROI file the shared arrays! 
Job Done for ROI- 120
Brain shape  (91, 109, 91)
Stored ROI file the shared arrays! 
Job Done for ROI- 138
Brain shape  (91, 109, 91)
Stored ROI file the shared arrays! 
Job Done for ROI- 103
Brain shape  (91, 109, 91)
Stored ROI file the shared arrays! 
Brain shape  (91, 109, 91)
Job Done for ROI- 112
Stored ROI file the shared arrays! 
Job Done for ROI- 94
Brain shape  (91, 109, 91)
Stored ROI file the shared arrays! 
Job Done for ROI- 76
Brain shape  (91, 109, 91)
Stored ROI file the shared arrays! 
Job Done for ROI- 85
Brain shape  (91, 109, 91)
Stored ROI fi

Stored ROI file the shared arrays! 
Job Done for ROI- 150
Brain shape  (91, 109, 91)
Stored ROI file the shared arrays! 
Job Done for ROI- 195
Brain shape  (91, 109, 91)
Stored ROI file the shared arrays! 
Job Done for ROI- 168
Brain shape  (91, 109, 91)
Stored ROI file the shared arrays! 
Job Done for ROI- 186
Brain shape  (91, 109, 91)
Stored ROI file the shared arrays! 
Job Done for ROI- 204
Brain shape  (91, 109, 91)
Stored ROI file the shared arrays! 
Job Done for ROI- 213
Brain shape  (91, 109, 91)
Stored ROI file the shared arrays! 
Job Done for ROI- 160
Brain shape  (91, 109, 91)
Brain shape  (91, 109, 91)
Stored ROI file the shared arrays! 
Stored ROI file the shared arrays! 
Job Done for ROI- 151
Job Done for ROI- 178
Brain shape  (91, 109, 91)
Stored ROI file the shared arrays! 
Job Done for ROI- 169
Brain shape  (91, 109, 91)
Stored ROI file the shared arrays! 
Brain shape  (91, 109, 91)
Job Done for ROI- 187
Stored ROI file the shared arrays! 
Job Done for ROI- 196
Brain s

In [18]:
# shared arrays are Array proxy objects so convert to np array to save them as brains
fdrcorrected_brain_4Dtensor = np.asarray(fdrcorrected_brain_4Dtensor)
fdrcorrected_brain_4Dtensor_mask = np.asarray(fdrcorrected_brain_4Dtensor_mask)
fdr_brain_voxel_list = np.asarray(fdr_brain_voxel_list)

# Save the Q values (fdr corrected values)
np.save('fdr_brain_voxel_list',fdr_brain_voxel_list)

# Saving the fdr corrected brains: 
brain_with_header = nib.Nifti1Image(fdrcorrected_brain_4Dtensor, affine=brain_data.affine,header = brain_data.header)
nib.save(brain_with_header,'fdr_corrected_brains_wo_z_standardardization.nii.gz')

brain_with_header = nib.Nifti1Image(fdrcorrected_brain_4Dtensor_mask, affine=brain_data.affine,header = brain_data.header)
nib.save(brain_with_header,'fdr_corrected_brains_mask_wo_z_standardardization.nii.gz')



In [19]:
np.min(fdrcorrected_brain_4Dtensor),np.max(fdrcorrected_brain_4Dtensor)

(0.0, 1.0)

In [20]:
res =(np.unique(np.where(fdrcorrected_brain_4Dtensor_mask[:,:,:,:] == 1)[3],  return_counts= True))

In [21]:
count_roi = zip(list(res[0]), list(res[1]))
set(count_roi)

{(4, 2),
 (6, 1),
 (18, 9),
 (26, 8),
 (38, 11),
 (42, 4),
 (60, 17),
 (61, 5),
 (73, 1),
 (76, 3),
 (77, 8),
 (83, 4),
 (92, 4),
 (144, 1),
 (156, 14),
 (165, 30),
 (166, 50),
 (167, 128),
 (169, 3),
 (171, 1),
 (172, 1),
 (173, 3),
 (179, 1),
 (201, 1),
 (213, 1),
 (227, 1),
 (232, 13),
 (236, 229),
 (241, 2)}

In [10]:
count_roi = zip(list(res[0]), list(res[1]))

In [11]:
set(count_roi)

{(8, 2),
 (18, 3),
 (19, 1),
 (24, 50),
 (26, 3),
 (33, 2),
 (40, 1),
 (41, 23),
 (45, 2),
 (46, 6),
 (47, 322),
 (60, 7),
 (61, 15),
 (72, 2),
 (74, 128),
 (80, 172),
 (81, 10),
 (83, 31),
 (86, 5),
 (87, 12),
 (89, 3),
 (92, 39),
 (97, 2),
 (99, 2),
 (100, 2),
 (103, 24),
 (107, 36),
 (114, 1),
 (120, 25),
 (121, 4),
 (122, 4),
 (144, 99),
 (156, 82),
 (157, 7),
 (158, 3),
 (160, 11),
 (166, 9),
 (167, 1),
 (171, 1),
 (173, 2),
 (177, 17),
 (181, 8),
 (201, 4),
 (226, 4),
 (227, 3),
 (252, 3),
 (264, 1)}

In [61]:
np.where(fdrcorrected_brain_4Dtensor_mask[:,:,:,47] == 1)

(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))

Observations:(ROI starts from 0) - Total Voxels  
ROI 47 (A11m_R, Medial Area 11 )           - 322 Voxels  - Precuneous ke saath correlated  
ROI 74 (A22c_L, Caudal Area 21 )           - 128 Voxels  
ROI 80 (A21c_L. Caudal Area 22)            - 172 Voxels  - Thalamus ke saath correlated  
ROI 144 (A40rv_L, Rostroventral area 40 )  - 99 Voxels  
ROI 156 (A1/2/3tonIa_L(Postcentral Gyrus)) - 82 Voxels  



ROI 47 is a part of Brodmann area 11. Brodmann area 11 is one of Brodmann's cytologically defined regions of the brain. It is involved in decision making[1] and processing rewards,[2] planning[citation needed], encoding new information into long-term memory,[3] and reasoning[citation needed].

It is ##Negatively correlated with Precuneous which is involved with episodic memory, visuospatial processing, reflections upon self, and aspects of consciousness.

My hypotheesis will be that they will under connected in Autistic people. Find out.



## To check if my hypothesis is correct:
* I create the fMRI volume of ROIs same as I did in do_fdr  

In [21]:
def create_t_value_file(t_value_file, brain_voxel_list):
    mask_data = nib.load(mask_file)
    mask = mask_data.get_data()
    
    # to use the deader of an anatomical file tht has float datatype. mask had uint datatype that made all the p-values zero
    brain_file = '/home1/varunk/data/ABIDE-BIDS-Preprocessed/NYU/sub-0050952/anat/sub-0050952_T1w.nii.gz'
    brain_data = nib.load(brain_file)
    
    
    # Two empty brain to store the fdr corrected values and mask (which voxel is correlated)
    brain = brain_data.get_data()
    
#     brain_corrected_mask = np.zeros((brain.shape))


    roi_number = int(brain_voxel_list[0]) - 1
    brain_voxel_list = brain_voxel_list[1:]
    
      
    
    x_dim, y_dim, z_dim = mask.shape

    brain_voxel_counter = 0
    for i in range(x_dim):
        for j in range(y_dim):
            for k in range(z_dim):
                if mask[i,j,k] == 1:
                    brain[i,j,k] = brain_voxel_list[brain_voxel_counter]
                    brain_voxel_counter = brain_voxel_counter + 1
                    

    
    
    if brain_voxel_counter == len(brain_voxel_list):
        
        print('Brain shape ',brain.shape)
        t_value_file[:,:,:,roi_number] = brain
        
        print('Stored ROI file the shared arrays! ')
        print("Job Done for ROI-",roi_number)
      
        return 1
    else:
        print("p-values fitting failed for ROI-",roi_number)
        return 0

In [35]:
X_t_vals = np.load('/home1/varunk/Autism-Connectome-Analysis-bids-related/significance_matrix_2_t_values.npy', mmap_mode='r')
# stack a column indentifying the roi value
X_t_vals = np.c_[1:X_t_vals.shape[0]+1,X_t_vals]
# num_ROIs = X.shape[0]
# Create pool of 8 workers

pool_inputs = [] #np.arange(number_of_ROIs)

ROIs = [47,80,74]
for roi in ROIs:
    pool_inputs.append(X_t_vals[roi,:])
    

In [36]:
# Read mask files:
mask_file = os.path.expandvars('$FSLDIR/data/standard/MNI152_T1_2mm_brain_mask.nii.gz')
mask_data = nib.load(mask_file)
mask = mask_data.get_data()

# read the file on which you want to write the p-values
brain_file = '/home1/varunk/data/ABIDE-BIDS-Preprocessed/NYU/sub-0050952/func/sub-0050952_task-rest_run-1_bold.nii.gz'
brain_data = nib.load(brain_file)
brain = brain_data.get_data()

# Edit header to store ROI correlation maps in t(4th) dimension

(brain_data.header).set_data_shape([91,109,91,num_ROIs])

In [37]:
%%time
# print (brain_data.header)


m = MyManager()
m.start()

t_value_file = m.np_zeros((brain_data.header.get_data_shape()))

func = partial(create_t_value_file, t_value_file)

pool = Pool(8)

data_outputs = pool.map(func, pool_inputs)

Brain shape  (91, 109, 91)
Brain shape  (91, 109, 91)
Brain shape  (91, 109, 91)
Stored ROI file the shared arrays! 
Stored ROI file the shared arrays! 
Stored ROI file the shared arrays! 
Job Done for ROI- 80
Job Done for ROI- 74
Job Done for ROI- 47
CPU times: user 16 ms, sys: 84 ms, total: 100 ms
Wall time: 2.26 s


In [38]:
t_value_file = np.asarray(t_value_file)

In [44]:
t_value_file[48,34,45,47]

-6.0584564208984375

## Result: As I thought, It is negative.
* Medial is under connected to precunious in Autistic individuals as compared to Healthy

In [52]:
fdrcorrected_brain_4Dtensor[29,29,48,39]

0.08949221670627594

In [53]:
test_mask = nib.load('fdr_corrected_brains_mask_2.nii.gz')

In [38]:
mask_brain_test = test_mask.get_data()

In [28]:
np.where(mask == 1)

(array([ 9,  9,  9, ..., 80, 80, 80]),
 array([46, 46, 46, ..., 52, 52, 53]),
 array([34, 35, 36, ..., 30, 31, 30]))

In [34]:
!fslhd fdr_corrected_brains.nii.gz;

ERROR: Could not open file
